In [122]:
# %load notebook_setup.py
#Use %load notebook_setup.py to load these commands at the top of your
#ipython notebook.  Includes running the std_libraries.py file. 

%run std_libraries.py
%load_ext autoreload
%autoreload 2
# %matplotlib 
# plt.ioff()
%matplotlib inline
#%matplotlib notebook

home directory is unset
y1000plus_tools dir is unset
y1000plus data dir is unset
what computer are you on? a = Ben's laptop, b = gpucluster, c = Ben's desktop, d = othera
home directory is C:\Users\BMH_work\
y1000plus_tools dir is C:\Users\BMH_work\github\y1000plus_tools\
y1000plus data dir is C:\Users\BMH_work\github\y1000plus_tools\genomes\y1000plus\
Importing y1000plus_tools.py
importing yeast_esr_exp
['C:\\Users\\BMH_work\\github\\y1000plus_tools\\scripts', 'C:\\Users\\BMH_work\\Anaconda3\\envs\\y1000_plus_env\\python37.zip', 'C:\\Users\\BMH_work\\Anaconda3\\envs\\y1000_plus_env\\DLLs', 'C:\\Users\\BMH_work\\Anaconda3\\envs\\y1000_plus_env\\lib', 'C:\\Users\\BMH_work\\Anaconda3\\envs\\y1000_plus_env', '', 'C:\\Users\\BMH_work\\Anaconda3\\envs\\y1000_plus_env\\lib\\site-packages', 'C:\\Users\\BMH_work\\Anaconda3\\envs\\y1000_plus_env\\lib\\site-packages\\win32', 'C:\\Users\\BMH_work\\Anaconda3\\envs\\y1000_plus_env\\lib\\site-packages\\win32\\lib', 'C:\\Users\\BMH_work\\Anaconda3\\e

In [4]:
y1000_species = pd.read_csv(y1000plus_dir + "y1000_species_table.csv", index_col=0)

#Select only clades of interest: 
#clades_of_interest = {'Saccharomycetaceae','Saccharomycodaceae','Phaffomycetaceae','CUG-Ser2','CUG-Ser1','Pichiaceae','CUG-Ala'}
clades_of_interest = {'Saccharomycetaceae', 'Saccharomycodaceae'}

outgroups = [ 'Wickerhamomyces anomalus','Cyberlindnera jadinii', 'Ascoidea rubescens','Komagataella pastoris','Debaryomyces hansenii', 'Candida albicans', 'Yarrowia lipolytica']
#here is a longer curated list of outgroups
#nonsacc_visualization_subset = pd.read_csv(y1000plus_dir + 'species_visualization_subset.csv')
#outgroup = 'Yarrowia lipolytica'

y1000_species_subset = y1000_species[(y1000_species['Major clade'].isin(clades_of_interest))  |  (y1000_species['Species name'].isin(outgroups))]


#outgroup = 'Hanseniaspora vineae'  #For EGO2/4 - only Sacch so use different outgroup
#y1000_species_subset = pd.concat([y1000_species_subset, y1000_species[y1000_species['Species name']==outgroup]])
#y1000_species_subset

genome_name_lookup = dict(zip(y1000_species_subset['species_names_fig2'], y1000_species_subset['original_genome_id']))



#Load Ohnolog data and make ancestor look up table that links the common name of the high gene to the ancestor. 
ohnologs_expression_sorted = pd.read_csv(yeast_esr_exp_path + os.path.normpath("expression_data/kl_sc_PKA_as_m24_r1g1_20181017/20200607_ohnologs_expression_sorted.csv"), index_col=0)
anc_lookup = dict(zip(ohnologs_expression_sorted['SC_common_name_high'], ohnologs_expression_sorted.index))


In [3]:
# nonsacc_visualization_subset = pd.read_csv(y1000plus_dir + 'species_visualization_subset.csv')
# species_subset = ( set(nonsacc_visualization_subset['original_genome_id']) | \
#                    set(y1000_species[y1000_species['Major clade']=='Saccharomycetaceae']['original_genome_id']) ) # | \
#                    #set(y1000_species[y1000_species['Species name']==outgroup]['original_genome_id'])
#                  #)
# y1000_species_subset = y1000_species[(y1000_species['original_genome_id'].isin(species_subset))]

In [30]:
#For input goi pair, get surrounding genenames from YGOB site.  
#Convention for this script is to put the high LFC gene name first 

goi_pair = 'EGO4_EGO2' #'GPM2_GPM3'  #GND2_GND1 #'EGO4_EGO2'
focus_gene = goi_pair.split('_')[0]

focus_anc = anc_lookup[focus_gene]

#Extract files using url
window = '8'

load_saved_synteny_window = True

if load_saved_synteny_window:
    synteny_window = pd.read_csv(yeast_esr_exp_path + os.path.normpath('expression_data/example_synteny/'+goi_pair + '_YGOB.csv'), index_col = 0)

else: 
    YGOB_query_url = 'http://ygob.ucd.ie/cgi/browser/tab.pl?ver=Latest&win=' + window + '&gene=' + focus_anc
    YGOB_query_response = requests.get(YGOB_query_url)

    soup = BeautifulSoup(YGOB_query_response.text)
    table = soup.find("table")

    output_rows = []
    for table_row in table.findAll('tr'):
        columns = table_row.findAll('td')
        output_row = []
        for column in columns:
            output_row.append(column.text)
        output_rows.append(output_row)

    synteny_window = pd.DataFrame(output_rows[2:],columns = output_rows[1])

    synteny_window.to_csv(yeast_esr_exp_path + os.path.normpath('expression_data/example_synteny/'+goi_pair + '_YGOB.csv'))


#synteny_row = synteny_window[synteny_window['Ancestor']==anc]
# print("\t".join(synteny_row.iloc[0,:].values) + '\n')
# f.write("\t".join(synteny_row.iloc[0,:].values) + '\n')

In [36]:
#Make K.lactis map from Kl genename to ygob name
spec = 'Klac'
orth_dir = os.path.normpath(yeast_esr_exp.data_processing_dir + 'ortholog_files_y1000'  ) + os.sep

klac_YGOB_to_y1000 = yeast_esr_exp.read_orth_lookup_table(spec + '_YGOB', spec + '_y1000', orth_dir)

#Make og lookups for S.cer genes
klac_lookup_fname = y1000plus_dir + os.path.normpath('id_lookups/kluyveromyces_lactis.csv')
klac_lookup = pd.read_csv(klac_lookup_fname, index_col=0)

In [37]:
kl_y1000ogid_lookup = {}
for kl_genename in list(set(synteny_window['K.lactis'])-{'---'}):
    kl_genename_y1000_options = klac_YGOB_to_y1000[kl_genename]
    if len(kl_genename_y1000_options)==1:
        if kl_genename_y1000_options[0] != 'NONE':
            y1000_no = kl_genename_y1000_options[0]
            kl_y1000ogid_lookup[kl_genename] = klac_lookup.loc[y1000_no,'y1000_id']
        else: 
            print('No sequence match for ' + kl_genename)
            kl_y1000ogid_lookup[kl_genename] = 'NONE'
    else: 
        kl_y1000ogid_lookup[kl_genename] = 'AMBIGUOUS'
        print('More than one sequence match for ' + kl_genename)
            

No sequence match for KLLA0C08437g


In [60]:
gois_sc = list((set(synteny_window['S.cerevisiae A']) | set(synteny_window['S.cerevisiae B']))-{'---'})

#Make og lookups for S.cer genes
scer_lookup_fname = y1000plus_dir + os.path.normpath('id_lookups/saccharomyces_cerevisiae.csv')

scer_lookup = pd.read_csv(scer_lookup_fname, index_col=0)
#target_y1000_id = scer_lookup.loc[gois_sc,'y1000_id']
y1000_ids = dict(scer_lookup.loc[gois_sc,'y1000_id'])
#y1000_id_list = [y1000_ids[goi] for goi in gois]

gois = list((set(kl_y1000ogid_lookup.values()) | set(y1000_ids.values())) - {'NONE'})

#Add an extra gene's OG - this OG should be included but wasn't in the same OG as S. cerevisiae and K. lactis.  
extra_spec_gene = ('naumovozyma_castellii', 'Seq_2431')
extra_spec_ids = pd.read_csv(y1000plus_dir + os.path.normpath('id_lookups/' + extra_spec_gene[0] + '.csv'))
extra_spec_y1000_lookup = dict(zip(extra_spec_ids['gene_id'], extra_spec_ids['y1000_id']))
extra_id = extra_spec_y1000_lookup[extra_spec_gene[1]]

gois.append(extra_id)


goi_og_lookup, og_genes_lookup = y1000plus_tools.make_og_genes_lookup(gois, y1000_species_subset)

more than one goi in same orthogroup :OG2262 found_genes: 
['50_1817', '110_1608']
more than one goi in same orthogroup :OG2411 found_genes: 
['110_3243', '110_1459', '50_1567']
more than one goi in same orthogroup :OG2918 found_genes: 
['110_2274', '50_4899']
more than one goi in same orthogroup :OG3003 found_genes: 
['50_739', '110_1780']
more than one goi in same orthogroup :OG3210 found_genes: 
['50_4534', '110_4843']
more than one goi in same orthogroup :OG3245 found_genes: 
['50_1876', '110_3759']
more than one goi in same orthogroup :OG3448 found_genes: 
['110_241', '50_4803']
more than one goi in same orthogroup :OG3502 found_genes: 
['110_2258', '50_532']
more than one goi in same orthogroup :OG3534 found_genes: 
['50_80', '110_815']
more than one goi in same orthogroup :OG4390 found_genes: 
['50_2219', '110_966']
more than one goi in same orthogroup :OG6958 found_genes: 
['50_4079', '110_700']


In [63]:
spec_og_search = ['S.cerevisiae A', 'S.cerevisiae B', 'K.lactis']


#Orthogroups found from S. cerevisiae and K. lactis orthologs of each gene may not be complete - this includes extra orthogroups. 
#It would be better to search for orthogroups based on genes from each species present in YGOB. Also need to have og present
#in og lookup so need to add the gene as a goi above.  

additional_ogs = {'EGO4_EGO2': [('Anc_6.348','OG38595')]  #found that EGO4_EGO2 had a small orthogroup containing naumovozyma_castellii_Seq_2431 and naumovozyma_dairenensis_Seq_3886
                 }

anc_genes = []
for anc in synteny_window['Ancestor']: 
    if anc != '---':
        anc_genes.append((anc, dict(synteny_window.loc[synteny_window['Ancestor']==anc, spec_og_search].iloc[0,:])))
# For Ancestor Column, Make list of tuple: species, genename.  

anc_genes_ogs = OrderedDict()
anc_genes_ogs_flat = OrderedDict()
for anc, gene_dict in anc_genes: 
    anc_og_dict = {}
    for spec in spec_og_search: 
        gene = gene_dict[spec]
        
        og = ''
        
        if gene !='---': 
            if 'S.cerevisiae' in spec: 
                y1000_id = y1000_ids[gene]
                og = goi_og_lookup[y1000_id]
            elif spec == 'K.lactis':
                y1000_id = kl_y1000ogid_lookup[gene]
                if not(y1000_id in {'NONE', 'AMBIGUOUS'}):
                    og = goi_og_lookup[y1000_id]
        anc_og_dict[spec] = og
    anc_genes_ogs[anc]  = anc_og_dict
    anc_genes_ogs_flat[anc] = list((set(anc_og_dict.values()) - {''}))


#Add additional ogs. 
if goi_pair in additional_ogs.keys():
    for anc, additional_og in additional_ogs[goi_pair]:
        anc_genes_ogs_flat[anc].append(additional_og)
    
anc_genes_ogs_flat



OrderedDict([('Anc_6.343', ['OG3448']),
             ('Anc_6.344', ['OG2262']),
             ('Anc_6.345', ['OG2411']),
             ('Anc_6.346', ['OG4390']),
             ('Anc_6.347', ['OG6695']),
             ('Anc_6.348', ['OG6958', 'OG10444', 'OG38595']),
             ('Anc_6.349', ['OG6956', 'OG22604']),
             ('Anc_6.350', ['OG2918']),
             ('Anc_6.351', ['OG3502']),
             ('Anc_6.352', ['OG3210']),
             ('Anc_6.353', ['OG3245']),
             ('Anc_6.354', ['OG3534']),
             ('Anc_6.355', ['OG3003'])])

In [123]:
#For a given example pair, find synteny of genes surrounding the ortholog in y1000 species not contained in YGOB.  


#Make list of post-WGH species
#set up data matrix based on time-calibrated species tree, and take subset of species based on species subset
tree_fname = y1000plus_dir + '332_2408OGs_time-calibrated_phylogeny_species-names_updated.newick'
t = Tree(tree_fname)
t.ladderize()

post_wgh_specs = t.get_common_ancestor(['Vanderwaltozyma_polyspora', 'Saccharomyces_cerevisiae']).get_leaves()
post_wgh_specs_names = [node.name for node in post_wgh_specs]
post_wgh_specs_names.remove('Saccharomyces_cerevisiae')
#For each species, load gtf, get surrounding genes, compare surrounding genes with orthologs (from ogs) of surrounding genes
bp_range = 20000  #Base pair range - goal is to get at least 8 CDS features

synteny_window_new_cols = synteny_window

for spec_fig2 in post_wgh_specs_names: 
    #spec_fig2 = 'Saccharomyces_arboricola'
    print(spec_fig2)
    focus_spec = genome_name_lookup[spec_fig2]
    synteny_window_new_cols = y1000plus_tools.find_spec_synteny(synteny_window_new_cols, focus_spec, focus_anc, genome_name_lookup, y1000_species_subset, anc_genes_ogs_flat, og_genes_lookup, bp_range)


Yueomyces_sinensis
Tetrapisispora_blattae
Vanderwaltozyma_polyspora
Tetrapisispora_iriomotensis
Tetrapisispora_phaffii
Tetrapisispora_fleetii
Tetrapisispora_namnaonensis
Candida_castellii
Nakaseomyces_bacillisporus
Candida_glabrata
More than one CDS - introns?.  N of CDS: 2
Candida_bracarensis
Nakaseomyces_delphensis
Candida_nivariensis
Saccharomyces_eubayanus
Saccharomyces_uvarum
Saccharomyces_arboricola
Saccharomyces_kudriavzevii
Saccharomyces_mikatae
Saccharomyces_paradoxus
Naumovozyma_castellii
Naumovozyma_dairenensis
Kazachstania_viticola
More than one CDS - introns?.  N of CDS: 2
Kazachstania_africana
Kazachstania_rosinii
Kazachstania_spencerorum
Kazachstania_kunashirensis
Kazachstania_turicensis
Kazachstania_martiniae
Kazachstania_intestinalis
Kazachstania_bromeliacearum
Kazachstania_naganishii
Kazachstania_taianensis
Kazachstania_yakushimaensis
Kazachstania_transvaalensis
Kazachstania_siamensis
Kazachstania_unispora
Kazachstania_aerobia
Kazachstania_solicola


In [113]:
synteny_window_new_cols.to_csv(yeast_esr_exp_path + os.path.normpath('expression_data/example_synteny/'+goi_pair + '.csv'))

In [235]:
synteny_window_new_cols.loc[:,['S.cerevisiae A', 'S.cerevisiae B','saccharomyces_arboricola_Seq_2087','saccharomyces_arboricola_Seq_2204']]

,S.cerevisiae A,S.cerevisiae B,saccharomyces_arboricola_Seq_2087,saccharomyces_arboricola_Seq_2204
0,---,YNR032C-A,---,---
1,---,YNR033W,---,---
2,---,---,NaN,NaN
3,YCR073W-A,YNR034W,---,Seq_3505
4,YCR075C,---,---,Seq_3359
5,---,---,NaN,NaN
6,---,---,---,---
7,---,---,NaN,NaN
8,YCR075W-A,YNR034W-A,Seq_2087,Seq_2204
9,YCR076C,---,---,---


In [296]:

#Replaced with functions

#Make list of post-WGH species
#set up data matrix based on time-calibrated species tree, and take subset of species based on species subset
tree_fname = y1000plus_dir + '332_2408OGs_time-calibrated_phylogeny_species-names_updated.newick'
t = Tree(tree_fname)
t.ladderize()

post_wgh_specs = t.get_common_ancestor(['Vanderwaltozyma_polyspora', 'Saccharomyces_cerevisiae']).get_leaves()
post_wgh_specs_names = [node.name for node in post_wgh_specs]
post_wgh_specs_names.remove('Saccharomyces_cerevisiae')
#For each species, load gtf, get surrounding genes, compare surrounding genes with orthologs (from ogs) of surrounding genes
bp_range = 20000  #Base pair range - goal is to get at least 8 CDS features


synteny_window_new_cols = synteny_window

for spec_fig2 in post_wgh_specs_names: 
    print(spec_fig2)
    focus_spec = genome_name_lookup[spec_fig2]
    #focus_spec = 'saccharomyces_arboricola'
    
    #'saccharomyces_uvarum',
    #  'saccharomyces_eubayanus',
    #  'saccharomyces_kudriavzevii',
    #  'saccharomyces_arboricola'

    #Get focus species genes that are present in focus ogs
    spec_og_id_lookup = dict(zip( y1000_species_subset['original_genome_id'], y1000_species_subset['spec_og_id']))
    spec_og_id = spec_og_id_lookup[focus_spec]
    focus_ogs = anc_genes_ogs_flat[focus_anc]

    og_genes = set()
    for og in focus_ogs: 
        og_genes = og_genes | og_genes_lookup[og]

    focus_spec_genes = []
    for og_gene in list(og_genes): 
        spec_og_id_test = int(og_gene.split('_')[0])
        if spec_og_id == spec_og_id_test:
            focus_spec_genes.append(og_gene)

    #Make full_gene lookup for focus spec
    focus_spec_lookup_fname = y1000plus_dir + os.path.normpath('id_lookups/' + focus_spec + '.csv')
    focus_spec_lookup = pd.read_csv(focus_spec_lookup_fname, index_col=0)
    y1000_id_gene_full_lookup = dict(zip(focus_spec_lookup['y1000_id'],focus_spec_lookup['gene_full']))        
    gene_full_y1000_id_lookup = dict(zip(focus_spec_lookup['gene_full'],focus_spec_lookup['y1000_id']))    
    y1000_id_gene_id_lookup = dict(zip(focus_spec_lookup['y1000_id'],focus_spec_lookup.index))    

    # Load GTF for given sequence: 
    gtf_dir = y1000plus_dir + "0_332yeast_genomes/332_genome_annotations/gtf/"
    db_fname = gtf_dir + 'gffutils_dbs/' + focus_spec + '.db'

    gtf_db = gffutils.FeatureDB(db_fname)


    # Make a dictionary of scaffold sizes
    genome_assembly_fname = y1000plus_dir + os.path.normpath('0_332yeast_genomes/332_genome_assemblies/' + focus_spec + '.fas')

    scaffold_sizes = {}
    for seq in SeqIO.parse(genome_assembly_fname, "fasta"):
        scaffold_sizes[seq.id] = len(seq.seq)


    #For each of these genes, find the surrounding genes
    for og_gene in focus_spec_genes: 
        #og_gene = '109_2086'
        focus_gene_id = y1000_id_gene_id_lookup[og_gene]

        gene_full = y1000_id_gene_full_lookup[og_gene]

        cursor = gtf_db.execute('select * from features where attributes like "%' + gene_full + '%"')
        all_features = cursor.fetchall()
        if len(all_features) == 0:
            print('No features found ' + gene_full + ' ' + focus_spec)  

        CDS_list = [feature for feature in all_features if feature['featuretype']=='CDS']
        if len(CDS_list)==1: 
            start = CDS_list[0]['start']
            end = CDS_list[0]['end']
        elif len(CDS_list)>1: 
            print('More than one CDS - introns?.  N of CDS: '+ str(len(CDS_list)))
            start = min([CDS['start'] for CDS in CDS_list])
            end = max([CDS['end'] for CDS in CDS_list])
        else: 
            raise AssertionError('No CDS found' + og_gene)
                  
        CDS = CDS_list[0]
        strand = CDS['strand']
        scaffold = CDS['seqid']

        # if strand=='+': 
        #     lower = start
        #     higher = end
        # elif strand == '-': 
        #     lower = end
        #     higher = start

        low = max(0,start-bp_range/2)
        high = min(scaffold_sizes[scaffold], end + bp_range/2)

        surrounding_CDS = list(gtf_db.region(region=(scaffold,low,high), completely_within=False, featuretype='CDS'))

        #surrounding_CDS info: (gene_full, gene_id, strand, start, end)
        surrounding_CDS_data = []
        for CDS_surr in surrounding_CDS:
            surrounding_CDS_data.append((CDS_surr.attributes['gene_id'][0],CDS_surr.id,CDS_surr.strand,CDS_surr.start,CDS_surr.end ))

        #Find OG and line that each gene is in
        anc_focus_spec_genes = OrderedDict([(anc,[]) for anc in anc_genes_ogs_flat.keys()])
        for (gene_full, gene_id, strand, start, end) in surrounding_CDS_data:
            y1000_id = gene_full_y1000_id_lookup[gene_full]
            for anc,og_list in anc_genes_ogs_flat.items():
                for og in og_list: 
                    og_genes = og_genes_lookup[og]
                    if y1000_id in og_genes:  #could put in a test to make sure this only happens once
                        anc_focus_spec_genes[anc].append(y1000_id_gene_id_lookup[y1000_id])
                        #could make triple dashes if len(anc_focus_spec_genes[anc]) == 0:
                #remove redundancies created by genes with introns
                anc_focus_spec_genes[anc]=list(set(anc_focus_spec_genes[anc]))
        #Merge into synteny window dataframe

        focus_gene_column = pd.DataFrame(pd.Series(anc_focus_spec_genes), columns = [focus_spec + '_' + focus_gene_id])
        synteny_window_new_cols = synteny_window_new_cols.merge(focus_gene_column, left_on='Ancestor', right_index = True, how='left')


Yueomyces_sinensis
Tetrapisispora_blattae
Vanderwaltozyma_polyspora
Tetrapisispora_iriomotensis
Tetrapisispora_phaffii
Tetrapisispora_fleetii
Tetrapisispora_namnaonensis
Candida_castellii
Nakaseomyces_bacillisporus
Candida_glabrata
Candida_bracarensis
Nakaseomyces_delphensis
Candida_nivariensis
Saccharomyces_eubayanus
Saccharomyces_uvarum
More than one CDS - introns?.  N of CDS: 8
Saccharomyces_arboricola
Saccharomyces_kudriavzevii
Saccharomyces_mikatae
More than one CDS - introns?.  N of CDS: 10
Saccharomyces_paradoxus
Naumovozyma_castellii
Naumovozyma_dairenensis
Kazachstania_viticola
Kazachstania_africana
More than one CDS - introns?.  N of CDS: 5
Kazachstania_rosinii
Kazachstania_spencerorum
Kazachstania_kunashirensis
Kazachstania_turicensis
Kazachstania_martiniae
Kazachstania_intestinalis
Kazachstania_bromeliacearum
Kazachstania_naganishii
Kazachstania_taianensis
Kazachstania_yakushimaensis
Kazachstania_transvaalensis
Kazachstania_siamensis
Kazachstania_unispora
Kazachstania_aerob